# Notes

# Imports

In [11]:
import numpy as np
np.set_printoptions(suppress=True)
np.seterr(divide='ignore', invalid='ignore')
import h5py
import time
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
import os
import sys
%matplotlib widget
%load_ext autoreload
%autoreload 2

# --- import functions for computing kinematic variables --- #
#sys.path.append('../tracking_code/lib/')
from FightDetector import FightDetector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# filepaths

In [3]:
main_load_folder = '/media/liam/hd1/fighting_data/tracking_results/'


# ----------------------#

# the path for data for the fight detector
clusterable_data_loadpath = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/clusterable_data.h5')

# the path to some cluster labels for the clusterable_data
clusterable_data_labels_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/clusterable_data_labels.h5')

# the path to the parameters used for the clustering
cluster_params_path = os.path.join( os.path.dirname(os.path.dirname(main_load_folder)) , 
                                      'fight_detection_data/cluster_params.h5')

# Load some data to test on

In [34]:
#  ---- prepare some unit-test filepaths, and the new pair1 filepaths ---- #

# We will load 4 tests and 4 new exps
numTestsExps = 4
paper_filepaths = glob.glob('/media/liam/hd1/fighting_data/tracking_results/*results.h5')
paper_filepaths.sort()
test_expNames = []
test_trajectories = []
for ii in range(numTestsExps):
    with h5py.File(paper_filepaths[ii], 'r') as hf:
        traj = hf['tracks_3D_smooth'][:]
        test_trajectories.append(traj)
        test_expNames.append( paper_filepaths[ii].split('/')[-1][:23] )
        

numNewExps = 4
pair1_filepaths = glob.glob('/media/liam/hd1/pair1_4_earlier_exps/*A.h5')
pair1_filepaths.sort()
trajectories = []
expNames = []
for ii in range(numNewExps):
    with h5py.File(pair1_filepaths[ii], 'r') as hf:
        traj = hf['tracks_3D_smooth'][:]
        trajectories.append(traj)
        expNames.append( pair1_filepaths[ii].split('/')[-1][:23] )


In [35]:
pair1_filepaths

['/media/liam/hd1/pair1_4_earlier_exps/FishTank20200124_150601_tracks_A.h5',
 '/media/liam/hd1/pair1_4_earlier_exps/FishTank20200219_161418_tracks_A.h5',
 '/media/liam/hd1/pair1_4_earlier_exps/FishTank20200228_160449_tracks_A.h5',
 '/media/liam/hd1/pair1_4_earlier_exps/FishTank20200401_162646_tracks_A.h5']

# Loading data needed for fight detector

In [51]:
# --- load arrays of interest ---- #

with h5py.File(clusterable_data_loadpath, 'r') as hf:
    expidx_and_decimated_wins_clusterable_data = hf['expidx_and_decimated_wins_clusterable_data'][:]
    prob_vectors_clusterable_data = hf['prob_vectors_clusterable_data'][:]

    
with h5py.File(clusterable_data_labels_path, 'r') as hf:
    cluster_labels = hf['cluster_labels'][:]
    fight_label_number = hf['fight_label_number'][()]
    
    
# load the clustering params dictionary
cluster_params = {}
with h5py.File(cluster_params_path, 'r') as hf:
    for param_key in hf.keys():
        try:
            param_value = hf[param_key][:]
        except:
            param_value = hf[param_key][()]
        cluster_params.update({param_key:param_value})
        
        


# Test the fight detector with paper trajectories

In [6]:
t0 = time.perf_counter()


# ---- make a fight detector ---- #
fd = FightDetector(cluster_params['dpp_bins'], cluster_params['g1_bins'], cluster_params['g2_bins'], 
                   cluster_params['window_size'], cluster_params['window_step'], cluster_params['skip_size'],
                   prob_vectors_clusterable_data, cluster_labels, fight_label_number, 
                   refining_skip_size=2, merge_gap_size_in_windows=5, min_region_window_size=5)



# ---- apply the all test exps ---- #

fight_detector_results = []

for ii,traj in enumerate(test_trajectories):
    expName = test_expNames[ii]
    fight_bouts_info = fd.detect_fight_bouts(traj)
    fight_detector_results.append(fight_bouts_info)
    print(expName, ' finished: ', time.perf_counter() - t0, ' s')

    
print('-----')
print()
tE = time.perf_counter()
print(tE-t0)

FishTank20200127_143538  finished:  575.6883355120663  s
FishTank20200129_140656  finished:  1142.8905114890076  s
FishTank20200130_153857  finished:  1467.622953023063  s
FishTank20200130_181614  finished:  1736.067417935934  s
-----

1736.0682709470857


In [7]:
fight_detector_results

[array([[385500., 427400.],
        [583500., 610400.],
        [643500., 805400.]]),
 array([[274500., 301400.],
        [385700., 409400.],
        [493700., 655400.],
        [700700., 718600.]]),
 array([[ 31500., 175400.],
        [301300., 319600.]]),
 array([[331300., 436400.]])]

In [37]:
fightBouts_info_path = os.path.join(main_load_folder, 'fightBouts.h5')

with h5py.File(fightBouts_info_path, 'r') as hf:
    refined_exps_fight_info = hf['refined_exps_fight_info'][:]
    
    
for ii in range(numTestsExps):
    exp_b_info = refined_exps_fight_info[refined_exps_fight_info[:,0]==ii]
    print(exp_b_info[:,1:3])
    print()

[[385500 427400]
 [583500 610400]
 [643500 805400]]

[[274500 301400]
 [385700 409400]
 [493700 655400]
 [700700 718600]]

[[ 31500 175400]
 [301300 319600]]

[[331300 436400]]



# TEMP - apply the fight detector to the new pair1 exps

In [19]:
t0 = time.perf_counter()


# ---- make a fight detector ---- #
fd = FightDetector(cluster_params['dpp_bins'], cluster_params['g1_bins'], cluster_params['g2_bins'], 
                   cluster_params['window_size'], cluster_params['window_step'], cluster_params['skip_size'],
                   prob_vectors_clusterable_data, cluster_labels, fight_label_number, 
                   refining_skip_size=2, merge_gap_size_in_windows=5, min_region_window_size=5)



# ---- apply the all test exps ---- #
fight_detector_results = []
for ii,traj in enumerate(trajectories):
    expName = expNames[ii]
    fight_bouts_info = fd.detect_fight_bouts(traj)
    fight_detector_results.append(fight_bouts_info)
    print(expName, ' finished: ', time.perf_counter() - t0, ' s')

    
print('-----')
print()
tE = time.perf_counter()
print(tE-t0)

FishTank20200124_150601  finished:  603.8128472529352  s
FishTank20200219_161418  finished:  945.2355972789228  s
FishTank20200228_160449  finished:  1243.008524490986  s
FishTank20200401_162646  finished:  1743.4432178710122  s
-----

1743.4440203770064


In [53]:
# combine the results for the experiments into one array
refined_exps_fight_info = []
for expIdx in range(len(fight_detector_results)):
    numBouts = fight_detector_results[expIdx].shape[0]
    if numBouts == 0:
        continue
    else:
        info_arr = np.full( (fight_detector_results[expIdx].shape[0],3), expIdx)
        info_arr[:, 1:] = fight_detector_results[expIdx]
        refined_exps_fight_info.append(info_arr)
refined_exps_fight_info = np.concatenate(refined_exps_fight_info, axis=0)

# add the durations column
refined_exps_fight_info = np.stack([refined_exps_fight_info[:,0],
                                     refined_exps_fight_info[:,1],
                                     refined_exps_fight_info[:,2],
                                     durations], axis=1)

refined_exps_fight_info

array([[     0,   3000,  75100,  72100],
       [     0, 118500, 186400,  67900],
       [     0, 203700, 219600,  15900],
       [     0, 552500, 572000,  19500],
       [     0, 610300, 626600,  16300],
       [     2,   3000,  78300,  75300]])

In [ ]:
# save the results
detectorSavePath = '/media/liam/hd1/pair1_4_earlier_exps/fightBouts.h5'
with h5py.File(detectorSavePath, 'w') as hf:
    hf.create_dataset('refined_exps_fight_info', data=refined_exps_fight_info)